# this is the api_key of openai model

In [50]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [51]:
%cd /content/drive/MyDrive/LLM_fact_checker

/content/drive/MyDrive/LLM_fact_checker


In [4]:
!pip install langchain-community unstructured langchain-openai openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━

# load the data set for retrieval system


In [52]:
import os
os.listdir()

['LLM_fact_checker.ipynb', 'sample_facts.csv']

In [53]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader

loader = UnstructuredCSVLoader(
    file_path="sample_facts.csv", mode="elements"
)
docs = loader.load()

In [20]:
docs

[Document(metadata={'source': 'sample_facts.csv', 'filename': 'sample_facts.csv', 'last_modified': '2025-11-20T17:43:46', 'text_as_html': "<table><tr><td>id</td><td>text</td><td>date</td><td>source</td></tr><tr><td>1</td><td>The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers.</td><td>2019-02-01</td><td>https://pib.gov.in</td></tr><tr><td>2</td><td>GST Council reduced the tax rate on electric vehicles in 2019 in certain categories.</td><td>2019-09-01</td><td>https://pib.gov.in</td></tr><tr><td>3</td><td>The Ministry of Health announced a national vaccination drive for COVID-19 starting Jan 2021.</td><td>2020-12-20</td><td>https://pib.gov.in</td></tr><tr><td>4</td><td>The central government approved 24x7 electricity supply projects in selected rural areas in 2018.</td><td>2018-07-10</td><td>https://pib.gov.in</td></tr><tr><td>5</td><td>The government increased MSP of wheat by 8% for the 2022 season.</td><td>2022-05-05</td><td>https://pib.gov.in</td

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [22]:
chunks[0:5]

[Document(metadata={'source': 'sample_facts.csv', 'filename': 'sample_facts.csv', 'last_modified': '2025-11-20T17:43:46', 'text_as_html': "<table><tr><td>id</td><td>text</td><td>date</td><td>source</td></tr><tr><td>1</td><td>The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers.</td><td>2019-02-01</td><td>https://pib.gov.in</td></tr><tr><td>2</td><td>GST Council reduced the tax rate on electric vehicles in 2019 in certain categories.</td><td>2019-09-01</td><td>https://pib.gov.in</td></tr><tr><td>3</td><td>The Ministry of Health announced a national vaccination drive for COVID-19 starting Jan 2021.</td><td>2020-12-20</td><td>https://pib.gov.in</td></tr><tr><td>4</td><td>The central government approved 24x7 electricity supply projects in selected rural areas in 2018.</td><td>2018-07-10</td><td>https://pib.gov.in</td></tr><tr><td>5</td><td>The government increased MSP of wheat by 8% for the 2022 season.</td><td>2022-05-05</td><td>https://pib.gov.in</td

# embeeding the dataset


In [23]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    model="text-embedding-3-large"
)

embeeding in the chunks in vector database


In [24]:
from langchain_community.vectorstores.faiss import FAISS
df_faiss = FAISS.from_documents(chunks, embeddings)
df_faiss

In [28]:
query = "The Indian government has announced free electricity to all farmers starting July 2025"
docs_faiss = df_faiss.similarity_search_with_score(query, k=5)
docs_faiss

[(Document(id='d3fdf60d-5ddf-4b13-897a-1266645b1f9c', metadata={'source': 'sample_facts.csv', 'filename': 'sample_facts.csv', 'last_modified': '2025-11-20T17:43:46', 'text_as_html': "<table><tr><td>id</td><td>text</td><td>date</td><td>source</td></tr><tr><td>1</td><td>The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers.</td><td>2019-02-01</td><td>https://pib.gov.in</td></tr><tr><td>2</td><td>GST Council reduced the tax rate on electric vehicles in 2019 in certain categories.</td><td>2019-09-01</td><td>https://pib.gov.in</td></tr><tr><td>3</td><td>The Ministry of Health announced a national vaccination drive for COVID-19 starting Jan 2021.</td><td>2020-12-20</td><td>https://pib.gov.in</td></tr><tr><td>4</td><td>The central government approved 24x7 electricity supply projects in selected rural areas in 2018.</td><td>2018-07-10</td><td>https://pib.gov.in</td></tr><tr><td>5</td><td>The government increased MSP of wheat by 8% for the 2022 season.</td><

In [29]:
# merge the docs_faiss
context_text = "\n\n".join([doc.page_content for doc , _score in docs_faiss])
context_text

"Government announced a 'Har Ghar Bijli' campaign in 2015 to extend rural electrification. 2015-01-18 https://pib.gov.in 28 A 2024 announcement allocated funds to upgrade rural distribution transformers in selected districts. 2024-07-02 https://pib.gov.in 29 The Department of Agriculture issued guidelines for precision farming subsidies in 2022. 2022-02-10 https://pib.gov.in 30 A 2018 circular limited free electricity schemes to demonstration projects in specific regions. 2018-08-09\n\nfarmers affected by floods in 2023. 2023-09-10 https://pib.gov.in 9 The electricity subsidy for domestic consumers in a particular state was implemented in 2024. 2024-01-12 state_govt_press_release 10 A 2017 policy allowed subsidised solar pumps for irrigation in select districts. 2017-11-20 https://pib.gov.in 11 The Ministry of Rural Development announced village electrification targets for 2022. 2021-12-01 https://pib.gov.in 12 In 2015 the government set up a loan-waiver scheme for\n\nhttps://pib.gov.i

In [31]:
prompt_template = f"""
You are a deterministic fact-checking assistant.
You must evaluate claims ONLY using the evidence provided by the retrieval system.

Claim: {query}

Evidence: {context_text}

Follow these strict rules:
1. Use ONLY the provided evidence. No external knowledge.
2. If the evidence partially matches but is not the same, mark "Unverifiable".
3. If evidence contradicts the claim, mark "False".
4. If evidence strongly supports the claim, mark "True".
5. Keep reasoning short and factual.
6. Output MUST be only valid JSON. No text before or after.

JSON format:
  "claim": "{query}",
  "verdict": "True | False | Unverifiable",
  "reasoning": "...",
  "relevant_evidence": [],
  "confidence": 0.0


"""

# now make the llm agent to facts check

In [32]:
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    model="gpt-4o-mini",
    temperature=0.1
)
response_text = model.invoke(prompt_template)

In [34]:
response_text.content

'{\n  "claim": "The Indian government has announced free electricity to all farmers starting July 2025",\n  "verdict": "False",\n  "reasoning": "The evidence does not support the claim of free electricity to all farmers starting July 2025. A 2018 circular limited free electricity schemes to demonstration projects in specific regions, and there is no mention of a new announcement for free electricity to all farmers.",\n  "relevant_evidence": [\n    "A 2018 circular limited free electricity schemes to demonstration projects in specific regions."\n  ],\n  "confidence": 1.0\n}'

In [36]:
from IPython.display import display, Markdown
display(Markdown(response_text.content))

{
  "claim": "The Indian government has announced free electricity to all farmers starting July 2025",
  "verdict": "False",
  "reasoning": "The evidence does not support the claim of free electricity to all farmers starting July 2025. A 2018 circular limited free electricity schemes to demonstration projects in specific regions, and there is no mention of a new announcement for free electricity to all farmers.",
  "relevant_evidence": [
    "A 2018 circular limited free electricity schemes to demonstration projects in specific regions."
  ],
  "confidence": 1.0
}

Now we check the our retrieval system is work so we made a preprocessing for CSV file and in future we cam make more according the dataset

# now the system is work fine we can modify or improve it by make is function and workflow for langgraph

In [54]:
def prepare_dataset(file_path):
    loader = UnstructuredCSVLoader(
    file_path="sample_facts.csv", mode="elements"
    )
    docs = loader.load()
    # split the text into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
        )
    chunks = splitter.split_documents(docs)

    #prepare the embedding
    embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    model="text-embedding-3-large"
    )
    # faiss index
    df_faiss = FAISS.from_documents(chunks, embeddings)
    return df_faiss

In [38]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 8.6 MB/s eta 0:00:00


now we make a workflow for the llm model for smooth run of agent

In [55]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated

class State(TypedDict):
    query: str # str is type of the query
    db_faiss: object
    context: str
    answer: str
    needs_retrieval: bool
    openai_api_key: str

In [56]:
# node of langgraph
def plan(state):
    query = state["query"]
    state["needs_retrieval"] = True
    return state

In [57]:
#retrieval
def retrieve(state):
    db_faiss = state["db_faiss"]
    query = state["query"]

    results = db_faiss.similarity_search_with_score(query, k=3)
    context = "\n\n".join([doc.page_content for doc , _score in results])
    state["context"] = context
    return state

In [58]:
# answer
def answer(state):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=state["openai_api_key"])
    query = state["query"]
    context_text = state["context"]

    prompt_template = f"""
You are a deterministic fact-checking assistant.
You must evaluate claims ONLY using the evidence provided by the retrieval system.

Claim: {query}

Evidence: {context_text}

Follow these strict rules:
1. Use ONLY the provided evidence. No external knowledge.
2. If the evidence partially matches but is not the same, mark "Unverifiable".
3. If evidence contradicts the claim, mark "False".
4. If evidence strongly supports the claim, mark "True".
5. Keep reasoning short and factual.
6. Output MUST be only valid JSON. No text before or after.

JSON format:
  "claim": "{query}",
  "verdict": "True | False | Unverifiable",
  "reasoning": "...",
  "relevant_evidence": [],
  "confidence": 0.0


"""
    response = llm.invoke(prompt_template)
    state["answer"] = response.content
    return state

In [59]:
graph = StateGraph(State)

graph.add_node("plan", plan)
graph.add_node("retrieve", retrieve)
graph.add_node("answer", answer)

graph.set_entry_point("plan")
graph.add_edge("plan", "retrieve")
graph.add_edge("retrieve", "answer")
graph.add_edge("answer", END)


agent = graph.compile()

check the workflow

In [66]:
db_faiss = prepare_dataset("sample_facts.csv")
query = "The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers."
result = agent.invoke({"query": query, "db_faiss": db_faiss, "openai_api_key": openai_api_key})

In [67]:
print(result["answer"])

{
  "claim": "The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers.",
  "verdict": "True",
  "reasoning": "The evidence directly states that the Government of India launched the PM-Kisan scheme providing ₹6000 per year to small farmers.",
  "relevant_evidence": [
    {
      "id": 1,
      "text": "The Government of India launched PM-Kisan scheme providing ₹6000 per year to small farmers.",
      "date": "2019-02-01",
      "source": "https://pib.gov.in"
    }
  ],
  "confidence": 1.0
}


It only give the answer based on the data set we give the retrieval system